In [1]:
import requests
import json
from PIL import Image
from io import BytesIO
import seaborn as sns
import pandas as pd
import numpy as np
from urllib.request import urlopen
from datetime import date, timedelta
import re
from time import sleep
import math

In [72]:
response = requests.get("https://iss.moex.com/iss/engines/stock/markets/bonds/securities.json")
# состав ответа https://iss.moex.com/iss/engines/stock/markets/bonds/boards/TQCB/securities/
# описание параметров https://iss.moex.com/iss/engines/stock/markets/bonds
response

<Response [200]>

In [73]:
rawData = pd.DataFrame(json.loads(response.text))

# вытаскиваю объект securities, формирую датасет, добавляю индексы
index = np.array([], int)

x = 0
for i in range (len(rawData.securities.data)):
  x += 1
  index = np.append(index,x)

bonds_securities = pd.DataFrame(rawData.securities.data, index, columns=rawData.securities.columns)
bonds_securities

SECID BOARDID   SHORTNAME  PREVWAPRICE  YIELDATPREVWAPRICE  \
1     SU26225RMFS1    PACT   ОФЗ 26225          NaN                0.00   
2     SU26248RMFS3    PACT   ОФЗ 26248          NaN                0.00   
3     SU26207RMFS9    SPOB   ОФЗ 26207          NaN                0.00   
4     SU26212RMFS9    SPOB   ОФЗ 26212      66.3445               12.71   
5     SU26218RMFS6    SPOB   ОФЗ 26218          NaN                0.00   
...            ...     ...         ...          ...                 ...   
2816  RU000A1094L0    TQRD  ФИНСИСТ1Р1       6.5100           892800.00   
2817  RU000A109791    TQRD  ГарИнв2P10      20.8300                0.00   
2818  RU000A10ADH7    TQRD  НафттрнБО6      35.2000               57.90   
2819  RU000A10AMP1    TQRD   МАГНУМ-01      37.9600               71.80   
2820  RU000A10B1M5    TQRD  НафттрнБО7      35.0800               45.13   

      COUPONVALUE  NEXTCOUPON  ACCRUEDINT  PREVPRICE  LOTSIZE  ...  LISTLEVEL  \
1           36.15  2025-11-19       15.69    68.0772        1  ...          1   
2           61.08  2025-12-03       21.81    89.5246        1  ...          1   
3           40.64  2026-02-04        0.45    93.0154        1  ...          1   
4           35.15  2026-01-21        3.09   104.1200        1  ...          1   
5           42.38  2025-09-24       31.44   111.8131        1  ...          1   
...           ...         ...         ...        ...      ...  ...        ...   
2816         0.00  2025-10-29        0.00     6.5100        1  ...          3   
2817         0.58  2026-01-01        0.00    20.5000        1  ...          3   
2818        24.66  2025-08-15        0.00    35.6800        1  ...          3   
2819        27.95  2025-08-12        0.00    38.0400        1  ...          3   
2820        23.01  2025-09-06        0.00    34.7800        1  ...          3   

     SECTYPE COUPONPERCENT OFFERDATE  SETTLEDATE  LOTVALUE  \
1          3          7.25      None  2025-08-08    1000.0   
2          3         12.25      None  2025-08-08    1000.0   
3          3          8.15      None  2025-08-08    1000.0   
4          3          7.05      None  2025-08-08    1000.0   
5          3          8.50      None  2025-08-08    1000.0   
...      ...           ...       ...         ...       ...   
2816       8           NaN      None  2025-08-08    1000.0   
2817       8          0.10      None  2025-08-08    1000.0   
2818       8         30.00      None  2025-08-08    1000.0   
2819       8         34.00      None  2025-08-08    1000.0   
2820       8         28.00      None  2025-08-08    1000.0   

      FACEVALUEONSETTLEDATE  CALLOPTIONDATE PUTOPTIONDATE DATEYIELDFROMISSUER  
1                       NaN            None          None                None  
2                       NaN            None          None                None  
3                    1000.0            None          None                None  
4                    1000.0            None          None                None  
5                    1000.0            None          None                None  
...                     ...             ...           ...                 ...  
2816                 1000.0            None          None                None  
2817                 1000.0            None          None                None  
2818                 1000.0            None          None                None  
2819                 1000.0            None          None                None  
2820                 1000.0            None          None                None  

[2820 rows x 43 columns]

In [74]:
# вытаскиваю объект marketdata, форимрую датасет, добавляю индексы
index = np.array([], int)

x = 0
for i in range (len(rawData.marketdata.data)):
  x += 1
  index = np.append(index,x)

bonds_marketdata = pd.DataFrame(rawData.marketdata.data, index, columns=rawData.marketdata.columns)
bonds_marketdata[['SECID','BOARDID','MARKETPRICE','DURATION','UPDATETIME']]

SECID BOARDID  MARKETPRICE  DURATION UPDATETIME
1     SU26225RMFS1    PACT       68.180         0   09:00:02
2     SU26248RMFS3    PACT       89.619         0   09:00:02
3     SU26207RMFS9    SPOB       93.782         0   09:00:10
4     SU26212RMFS9    SPOB       88.239         0   09:00:10
5     SU26218RMFS6    SPOB       80.011         0   09:00:10
...            ...     ...          ...       ...        ...
2816  RU000A1094L0    TQRD        5.630         0   10:39:00
2817  RU000A109791    TQRD       20.830         0   10:39:26
2818  RU000A10ADH7    TQRD       35.200         0   10:39:32
2819  RU000A10AMP1    TQRD       37.960         0   10:39:38
2820  RU000A10B1M5    TQRD       35.030         0   10:39:36

[2820 rows x 5 columns]

In [75]:
# склейка bonds_securities и bonds_marketdata для получения всех необходимых параметров 
bonds = pd.merge(bonds_securities,bonds_marketdata[['SECID','BOARDID','YIELD','LAST','DURATION','VOLTODAY']],how = 'left')
bonds = bonds[['SECNAME','SECID','BOARDID','LAST','LOTVALUE','YIELD','DURATION','OFFERDATE','MATDATE','COUPONPERCENT','ACCRUEDINT','VOLTODAY']]
# bonds
bonds.sort_values(by=['DURATION','LAST'],ascending=[False,False])

SECNAME         SECID BOARDID   LAST   LOTVALUE  \
2095              ОВОЗ РФ 12840119V  RU000A10A844    TQCB    NaN  200000.00   
2098              ОВОЗ РФ 12840118V  RU000A10A877    TQCB    NaN  200000.00   
2091              ОВОЗ РФ 12840117V  RU000A10A802    TQCB    NaN  200000.00   
91         АКБ ПАО "ПЕРЕСВЕТ" БО-03  RU000A0JVM32    TQCB  39.99    1000.00   
1096  Газпром капитал ООО БЗО26-1-Д  RU000A105QW3    TQCB  84.75    1000.00   
...                             ...           ...     ...    ...        ...   
2782                Сбер CIB-СО-718  RU000A109YZ2    TQOY    NaN    1000.00   
2783                Сбер CIB-СО-719  RU000A109Z01    TQOY    NaN    1000.00   
2792   Концессии водснаб Саратов 01  RU000A0ZZWQ8    TQRD    NaN     692.32   
2793  Концессии водсн Саратов БО-01  RU000A1018G9    TQRD    NaN     750.01   
2810          Фабрика ФАВОРИТ БО-01  RU000A106ZZ5    TQRD    NaN    1000.00   

      YIELD  DURATION   OFFERDATE     MATDATE  COUPONPERCENT     ACCRUEDINT  \
2095   6.46      4562        None  2047-06-23          5.250  105251.212500   
2098   6.47      4000        None  2043-09-16          5.875  371664.685408   
2091   6.51      3838        None  2042-04-04          5.625  310741.675000   
91    14.90      3604        None  2035-06-21          0.510      42.320000   
1096   5.59      3560        None  0000-00-00          4.599     122.692842   
...     ...       ...         ...         ...            ...            ...   
2782   0.00         0        None  2027-05-11          0.010       0.000000   
2783   0.00         0        None  2027-06-07          0.010       0.000000   
2792   0.00         0        None  2033-12-02         23.000       0.000000   
2793   0.00         0  2033-12-08  2033-12-08         24.500       0.000000   
2810   0.00         0        None  2027-09-19            NaN       0.000000   

      VOLTODAY  
2095         0  
2098         0  
2091         0  
91           5  
1096        17  
...        ...  
2782         0  
2783         0  
2792         0  
2793         0  
2810         0  

[2820 rows x 12 columns]

In [6]:
# Функция для получния кредитного рейтинга
def get_creadit_rating(response):
    try:
        credit_rating_match = re.search('Кредитный рейтинг',response.text) # поиск по заданной строке
        if not credit_rating_match:
            credit_rating_value = 'Not rating' # если строка не найдена, выход с возвратом значения 'Not rating' - у всяких госбондов обычно нет никакого рейтинга на смартлаб
            return credit_rating_value
        credit_rating_value_match = re.search('A|B|C|D|\t\-' ,response.text[credit_rating_match.end():]) # поиск значения рейтинг дальше по тексту (от позиции, где заканчивается строка "Кредитный рейтинг")
        credit_rating_value = response.text[credit_rating_match.end()+credit_rating_value_match.start()] # забирает первый символ рейтинга
        index = credit_rating_match.end()+credit_rating_value_match.start() + 1 # определяет стартовое значение для цикла (позция следующего символа после первого в рейтинге)
        for i in range(index,index+10):
          if re.match('\n',response.text[i]): # если обнаружен конец строки, итерация прерывается
              break
          if (credit_rating_value == '\t' and re.match('\-',response.text[i])): # если первый символ в рейтинге \t и следующий "-"
              credit_rating_value = response.text[i] # вместо рейтинга передается прочерк (скорее всего рейтинг отозван или типа того)
          else:
              credit_rating_value = credit_rating_value + response.text[i] # если рейтинг из нескольких символов собирается полное значение
    except Exception as e:
        print(f"Ошибка получении рейтингов: {e}" + isin) # если не удалось выполнить запрос для какого-то isin (ошибка выполнения запроса)

    return credit_rating_value # возвращается итоговое значение рейтинга

In [7]:
# Функция для получния необходимости квалификации для покупки
def get_qualification_need(response):
    try:
        qualification_need = re.search('Только для квалов?',response.text) # поиск по заданной строке
        qualification_need_match = re.search('Да|Нет' ,response.text[qualification_need.end():]) # поиск значения квалификации дальше по тексту (от позиции, где заканчивается строка "Только для квалов?")
        qualification_need_value = qualification_need_match.group(0)
    except Exception as e:
        print(f"Ошибка получении квалификации: {e}" + isin) # если не удалось выполнить запрос для какого-то isin (ошибка выполнения запроса)

    return qualification_need_value # возвращается итоговое значение необходимости квалификации для покупки

In [8]:
# подключение к БД
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [9]:
# Создание БД. Если такая есть, то подключение.
connection = create_connection("bonds.db")

Connection to SQLite DB successful


In [10]:
# функция для выполнения SQL запросов
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [ ]:
create_bonds_table = """
CREATE TABLE IF NOT EXISTS bonds (
  isin TEXT NOT NULL,
  board_id TEXT NOT NULL,
  short_name TEXT NOT NULL,
  credit_rating TEXT,
  CONSTRAINT id PRIMARY KEY (isin, board_id)
);
"""

In [ ]:
# грузит подготовленные значения в БД (нужна при создании БД)
bonds_for_db.to_sql(name = 'bonds', con = connection, if_exists='append', index=False)

In [10]:
read_bonds_data = """
SELECT * FROM bonds;
"""

In [13]:
execute_query(connection, read_bonds_data)

Query executed successfully


In [76]:
# Чтение данных из базы для получения ранее загруженных рейтингов
bonds_db = pd.read_sql(read_bonds_data, connection)
bonds_db = bonds_db.rename(columns = {'isin':'SECID','board_id':'BOARDID','short_name':'SHORTNAME','credit_rating':'RATING','qual':'Qual'})
bonds_db

SECID BOARDID                     SHORTNAME      RATING Qual
0     SU26225RMFS1    PACT         ОФЗ-ПД 26225 10/05/34  Not rating  Нет
1     SU26248RMFS3    PACT         ОФЗ-ПД 26248 16/05/40  Not rating  Нет
2     SU26207RMFS9    SPOB         ОФЗ-ПД 26207 03/02/27  Not rating  Нет
3     SU26212RMFS9    SPOB         ОФЗ-ПД 26212 19/01/28  Not rating  Нет
4     SU26218RMFS6    SPOB         ОФЗ-ПД 26218 17/09/31  Not rating  Нет
...            ...     ...                           ...         ...  ...
2815  RU000A1094L0    TQRD     ФИНАНСОВЫЕ СИСТЕМЫ БО-П01           D  Нет
2816  RU000A109791    TQRD  ФПК Гарант-Инвест БО 002Р-10           -  Нет
2817  RU000A10ADH7    TQRD     Нафтатранс плюс ООО БО-06          B+  Нет
2818  RU000A10AMP1    TQRD                  МАГНУМОЙЛ-01          B+  Нет
2819  RU000A10B1M5    TQRD     Нафтатранс плюс ООО БО-07          B+  Нет

[2820 rows x 5 columns]

In [77]:
# добавление рейтинга к списку облигаций с moex (склейка bonds_securities и bonds_marketdata)
bonds = pd.merge(bonds,bonds_db[['SECID','BOARDID','RATING','Qual']],how = 'left')
bonds

SECNAME         SECID BOARDID   LAST  LOTVALUE  \
0            ОФЗ-ПД 26225 10/05/34  SU26225RMFS1    PACT    NaN    1000.0   
1            ОФЗ-ПД 26248 16/05/40  SU26248RMFS3    PACT    NaN    1000.0   
2            ОФЗ-ПД 26207 03/02/27  SU26207RMFS9    SPOB    NaN    1000.0   
3            ОФЗ-ПД 26212 19/01/28  SU26212RMFS9    SPOB    NaN    1000.0   
4            ОФЗ-ПД 26218 17/09/31  SU26218RMFS6    SPOB    NaN    1000.0   
...                            ...           ...     ...    ...       ...   
2815     ФИНАНСОВЫЕ СИСТЕМЫ БО-П01  RU000A1094L0    TQRD   9.11    1000.0   
2816  ФПК Гарант-Инвест БО 002Р-10  RU000A109791    TQRD  21.50    1000.0   
2817     Нафтатранс плюс ООО БО-06  RU000A10ADH7    TQRD  36.00    1000.0   
2818                  МАГНУМОЙЛ-01  RU000A10AMP1    TQRD  37.70    1000.0   
2819     Нафтатранс плюс ООО БО-07  RU000A10B1M5    TQRD  34.95    1000.0   

      YIELD  DURATION OFFERDATE     MATDATE  COUPONPERCENT  ACCRUEDINT  \
0     14.05         0      None  2034-05-10           7.25       15.69   
1     14.42         0      None  2040-05-16          12.25       21.81   
2     13.93         0      None  2027-02-03           8.15        0.45   
3      5.30         0      None  2028-01-19           7.05        3.09   
4      6.24         0      None  2031-09-17           8.50       31.44   
...     ...       ...       ...         ...            ...         ...   
2815   0.00         0      None  2027-07-28            NaN        0.00   
2816   0.00         0      None  2030-07-30           0.10        0.00   
2817   0.00         0      None  2027-12-03          30.00        0.00   
2818   0.00         0      None  2027-12-30          34.00        0.00   
2819   0.00         0      None  2029-02-17          28.00        0.00   

      VOLTODAY      RATING Qual  
0            0  Not rating  Нет  
1            0  Not rating  Нет  
2            0  Not rating  Нет  
3            0  Not rating  Нет  
4            0  Not rating  Нет  
...        ...         ...  ...  
2815       168           D  Нет  
2816       706           -  Нет  
2817       206          B+  Нет  
2818       157          B+  Нет  
2819       359          B+  Нет  

[2820 rows x 14 columns]

In [35]:
# Проверка на наличие записей без рейтинга
# Находит записи: ~isna() с рейтингом, isna() - без рейтинга
condition = (bonds['RATING'].isna()) & (bonds['BOARDID'] == 'TQCB')
# condition = (bonds['Qual'].isna())
bonds_not_rating = bonds.loc[condition][['SECID','RATING','SECNAME']]
bonds_not_rating

Empty DataFrame
Columns: [SECID, RATING, SECNAME]
Index: []

In [26]:
# Находит записи: ~isna() с квалификацией/рейтингом, isna() - без квалификации/рейтингом
condition = (bonds['Qual'].isna()) | (bonds['RATING'].isna()) # условия для отбора облигаций без квалификации/рейтинга
bonds_not_qual_or_rating = bonds.loc[condition][['SECID','RATING','SECNAME','Qual']] # создание списка облигаций, удовлетворяющих условию
isin = ''
for index, row in bonds_not_qual_or_rating.iterrows():
    isin = bonds_not_qual_or_rating.loc[index]['SECID'] # присваивает переменной isin очередного элемента
    url = "https://smart-lab.ru/q/bonds/" + isin # формироние url
    response = requests.get(url) # запрос
    bonds.loc[index, 'Qual'] = get_qualification_need(response) # присваивает полученное значение квалификации очередному элементу
    bonds.loc[index, 'RATING'] = get_creadit_rating(response) # присваивает полученное значение рейтинга очередному элементу
    # sleep(4) # таймер нужен, когда требуется обновление для большого количества записей

In [22]:
# Для обновления рейтингов по компаниям
new_rate_list = bonds[bonds['SECNAME'].str.contains('РЕАТОРГ')]
isin = ''
for index, row in new_rate_list.iterrows():
    isin = new_rate_list.loc[index]['SECID'] # присваивает переменной isin очередного элемента
    bonds.loc[index, 'RATING'] = 'D'

bonds[bonds['SECNAME'].str.contains('РЕАТОРГ')]

SECNAME         SECID BOARDID  LAST  LOTVALUE  YIELD  DURATION  \
1016  РЕАТОРГ 001Р-01  RU000A105BP9    TQCB  None     150.0    0.0         0   
2796  РЕАТОРГ 001Р-01  RU000A105BP9    TQRD  None     150.0    0.0         0   

     OFFERDATE     MATDATE  COUPONPERCENT  ACCRUEDINT  VOLTODAY RATING Qual  
1016      None  2025-10-21           16.0         0.0         0      D  Нет  
2796      None  2025-10-21           16.0         0.0         0      D  NaN

In [97]:
bonds[bonds['SECNAME'].str.contains('Русская Контейнерная Компания')]

SECNAME         SECID BOARDID   LAST  LOTVALUE  \
2800  Русская Контейнерная Компания  RU000A105VP7    TQRD  35.13     750.0   

      YIELD  DURATION OFFERDATE     MATDATE  COUPONPERCENT  ACCRUEDINT  \
2800    0.0         0      None  2026-02-17           19.0         0.0   

      VOLTODAY RATING Qual  
2800      1682     CC  Нет

In [24]:
bonds.loc[bonds['SECID'] == 'RU000A10CAQ0', 'RATING'] = 'BB-'

In [27]:
bonds_for_db = bonds[['SECID','BOARDID','SECNAME','RATING','Qual']].rename(columns = {'SECID':'isin','BOARDID':'board_id','SECNAME':'short_name','RATING':'credit_rating','Qual':'qual'})
bonds_for_db

isin board_id                    short_name credit_rating qual
0     SU26225RMFS1     PACT         ОФЗ-ПД 26225 10/05/34    Not rating  Нет
1     SU26248RMFS3     PACT         ОФЗ-ПД 26248 16/05/40    Not rating  Нет
2     SU26207RMFS9     SPOB         ОФЗ-ПД 26207 03/02/27    Not rating  Нет
3     SU26212RMFS9     SPOB         ОФЗ-ПД 26212 19/01/28    Not rating  Нет
4     SU26218RMFS6     SPOB         ОФЗ-ПД 26218 17/09/31    Not rating  Нет
...            ...      ...                           ...           ...  ...
2815  RU000A1094L0     TQRD     ФИНАНСОВЫЕ СИСТЕМЫ БО-П01             D  Нет
2816  RU000A109791     TQRD  ФПК Гарант-Инвест БО 002Р-10             -  Нет
2817  RU000A10ADH7     TQRD     Нафтатранс плюс ООО БО-06            B+  Нет
2818  RU000A10AMP1     TQRD                  МАГНУМОЙЛ-01            B+  Нет
2819  RU000A10B1M5     TQRD     Нафтатранс плюс ООО БО-07            B+  Нет

[2820 rows x 5 columns]

In [28]:
# Обновляю БД с заменой значений (добавлись облигации без рейтинга). С заменой значений даже лучше, ведь что-то погасилось. Так не нужно отслеживать.
bonds_for_db.to_sql(name = 'bonds', con = connection, if_exists = 'replace', index = False)

2820

In [ ]:
# Убираю бонды дефолтников из общего списка
# bonds[bonds['SECNAME'].str.contains(r'Система')].sort_values(by=['YIELD','DURATION','LAST'],ascending=[False,False,True])
black_list = ["ПЕРЕСВЕТ","Завод КЭС","НИКА","РКК","ЧИСТАЯ ПЛАНЕТА","Пионер-Лизинг","Ультра","Техпорт","Кузина","ФАВОРИТ","Росгеология","Ситиматик-Югра",
       "Гарант-Инвест","Феррони","Селлер","ДОМОДЕДОВО","Борец","ФИНАНСОВЫЕ СИСТЕМЫ","ТГК-14","Мосрегионлифт","Нафтатранс плюс","МАГНУМОЙЛ","ГазТрансСнаб",
       "ЖКХ РС","С-ПРИНТ","РЕАТОРГ","Русская Контейнерная Компания"]
# bonds[bonds['SECNAME'].str.contains('ЖКХ РС')]
bonds_black_list = pd.DataFrame()
for str in black_list:
    bonds_black_list = pd.concat([bonds_black_list,bonds[bonds['SECNAME'].str.contains(str)]])

bonds = bonds[~bonds['SECID'].isin(bonds_black_list['SECID'])]
bonds

SECNAME         SECID BOARDID   LAST  LOTVALUE  \
0              ОФЗ-ПД 26225 10/05/34  SU26225RMFS1    PACT    NaN   1000.00   
1              ОФЗ-ПД 26248 16/05/40  SU26248RMFS3    PACT    NaN   1000.00   
2              ОФЗ-ПД 26207 03/02/27  SU26207RMFS9    SPOB    NaN   1000.00   
3              ОФЗ-ПД 26212 19/01/28  SU26212RMFS9    SPOB    NaN   1000.00   
4              ОФЗ-ПД 26218 17/09/31  SU26218RMFS6    SPOB    NaN   1000.00   
...                              ...           ...     ...    ...       ...   
2792    Концессии водснаб Саратов 01  RU000A0ZZWQ8    TQRD    NaN    692.32   
2793   Концессии водсн Саратов БО-01  RU000A1018G9    TQRD    NaN    750.01   
2800   Русская Контейнерная Компания  RU000A105VP7    TQRD  35.13    750.00   
2801  Рус Контейнерная Компания БО02  RU000A105ZW4    TQRD  22.05   1000.00   
2804  Рус Контейнерная Компания БО03  RU000A106GZ5    TQRD  20.79   1000.00   

      YIELD  DURATION   OFFERDATE     MATDATE  COUPONPERCENT  ACCRUEDINT  \
0     14.05         0        None  2034-05-10           7.25       15.69   
1     14.42         0        None  2040-05-16          12.25       21.81   
2     13.93         0        None  2027-02-03           8.15        0.45   
3      5.30         0        None  2028-01-19           7.05        3.09   
4      6.24         0        None  2031-09-17           8.50       31.44   
...     ...       ...         ...         ...            ...         ...   
2792   0.00         0        None  2033-12-02          23.00        0.00   
2793   0.00         0  2033-12-08  2033-12-08          24.50        0.00   
2800   0.00         0        None  2026-02-17          19.00        0.00   
2801   0.00         0        None  2027-03-18          18.00        0.00   
2804   0.00         0        None  2028-06-29          16.50        0.00   

      VOLTODAY      RATING Qual  
0            0  Not rating  Нет  
1            0  Not rating  Нет  
2            0  Not rating  Нет  
3            0  Not rating  Нет  
4            0  Not rating  Нет  
...        ...         ...  ...  
2792         0           -  Нет  
2793         0           -  Нет  
2800      1682          CC  Нет  
2801       591          CC  Нет  
2804      2100          CC  Нет  

[2755 rows x 14 columns]

In [79]:
# Получаю список лучших бумаг
condition = (bonds['DURATION'] <= 220) & (bonds['DURATION'] >= 30) & (bonds['YIELD'] >= 19) & (bonds['LAST'] <= 101.5) & (bonds['RATING'] != '-') & (bonds['RATING'] != 'Not rating') & (bonds['Qual'] != 'Да')
bonds_best = bonds.loc[condition].sort_values(by=['YIELD','DURATION','LAST'],ascending=[False,False,True])
bonds_best.head(30)

SECNAME         SECID BOARDID    LAST  LOTVALUE  \
1350                 АПРИ БО-002Р-03  RU000A106WZ2    TQCB   98.52   1000.00   
1495                 АПРИ БО-002Р-04  RU000A107FZ5    TQCB  100.70   1000.00   
1576                Брусника 002Р-02  RU000A107UU5    TQCB   98.61   1000.00   
2220              РДВ ТЕХНОЛОДЖИ 1P1  RU000A10ANT1    TQCB  101.32   1000.00   
1426             СОБИ-ЛИЗИНГ 001Р-03  RU000A1078H1    TQCB   94.76    490.00   
774        Регион-Продукт БО 001Р-02  RU000A103NE3    TQCB   92.18    625.00   
1161     Глобал Факторинг БО-02-001P  RU000A1060U2    TQCB   95.19    560.00   
1029                 АПРИ БО-002Р-01  RU000A105DS9    TQCB  100.36   1000.00   
1216             СОБИ-ЛИЗИНГ 001Р-02  RU000A106AK0    TQCB   95.68    310.00   
1153           БИЗНЕС АЛЬЯНС 001P-01  RU000A105YQ9    TQCB   94.14   1000.00   
1098                   Агротек БО-01  RU000A105QZ6    TQCB   96.60   1000.00   
822   НПП Моторные технологии001Р-03  RU000A103U69    TQCB   92.48    625.00   
1376           ЛК Роделен БО 002P-01  RU000A107076    TQCB   96.74   1000.00   
2237               МОНОПОЛИЯ 001P-03  RU000A10ARS4    TQCB  101.25   1000.00   
1551               ГК Самолет БО-П13  RU000A107RZ0    TQCB   96.20   1000.00   
1417                 МК Лизинг БO-01  RU000A107720    TQCB   96.91    833.50   
914               Транс-Миссия БО-01  RU000A104K11    TQCB   99.95    780.00   
718             Лизинг-Трейд 001P-03  RU000A1034X1    TQCB   95.65    750.10   
1293                       ПИМ БО-02  RU000A106NT4    TQCB   95.47   1000.00   
1391              АРЕНЗА-ПРО 001P-03  RU000A107217    TQCB   98.38   1000.00   
710             ТЕХНО Лизинг 001P-04  RU000A1032X5    TQCB   93.70    900.00   
1183                   АБЗ-1 001P-04  RU000A1065B1    TQCB   96.29    505.00   
1130           Эталон-Финанс 002Р-01  RU000A105VU7    TQCB   95.72   1000.00   
785          Эталон-Финанс АО БО-П03  RU000A103QH9    TQCB   92.73    560.00   
2366        Совкомбанк Лизинг БО-П09  RU000A10B3W0    TQCB   99.96    874.90   
1165       Хайтэк-Интеграция 001P-01  RU000A106151    TQCB   96.71   1000.00   
1150           Страна Девелопмент 02  RU000A105YF2    TQCB   96.36   1000.00   
1560              АРЕНЗА-ПРО 001P-04  RU000A107ST1    TQCB   97.36   1000.00   
827               МСБ-Лизинг 002P-05  RU000A103VD8    TQCB   95.17    388.84   
1232              МСБ-Лизинг 003P-01  RU000A106D18    TQCB   96.81    416.62   

      YIELD  DURATION   OFFERDATE     MATDATE  COUPONPERCENT  ACCRUEDINT  \
1350  39.44        52  2025-09-30  2027-09-14          24.00       34.19   
1495  33.75       148  2026-01-13  2027-12-17          32.00       42.96   
1576  33.12        41  2025-09-18  2027-03-28          16.25       10.24   
2220  30.54       174  2026-02-05  2029-07-26          30.50        6.68   
1426  30.30       211        None  2026-11-05          17.50        6.11   
774   29.95       192        None  2026-09-03          11.50       12.60   
1161  29.58       185        None  2026-09-08          16.50        6.08   
1029  29.50       101  2025-11-18  2026-11-03          28.00        2.30   
1216  29.09       140        None  2026-05-14          14.30        2.55   
1153  28.39       206        None  2026-03-12          14.85       23.19   
1098  27.90       139        None  2026-04-09          16.00       12.71   
822   27.45       219        None  2026-09-29          11.75        7.65   
1376  26.59       152  2026-01-12  2028-09-13          15.75        3.45   
2237  26.37       161        None  2026-01-25          26.50        7.26   
1551  25.61       173  2026-02-03  2027-01-24          14.75        2.42   
1417  25.52       217        None  2026-11-10          17.65        5.24   
914   25.42       151        None  2026-01-27          22.75        3.89   
718   25.21       140        None  2026-04-30          11.00        1.13   
1293  24.91       209        None  2026-07-30          14.50        3.18   
1391  24.73        89  2025-

In [80]:
condition = (bonds_best['RATING'] != 'BB') & (bonds_best['RATING'] != 'BB-') & (bonds_best['RATING'] != 'BB+') & (bonds_best['RATING'] != 'B+') & (bonds_best['RATING'] != 'B') & (bonds_best['RATING'] != 'B-')
bonds_best = bonds_best.loc[condition].sort_values(by=['YIELD','DURATION','LAST'],ascending=[False,False,True])
bonds_best.head(50)

SECNAME         SECID BOARDID    LAST  LOTVALUE  \
1350                 АПРИ БО-002Р-03  RU000A106WZ2    TQCB   98.52   1000.00   
1495                 АПРИ БО-002Р-04  RU000A107FZ5    TQCB  100.70   1000.00   
1576                Брусника 002Р-02  RU000A107UU5    TQCB   98.61   1000.00   
1029                 АПРИ БО-002Р-01  RU000A105DS9    TQCB  100.36   1000.00   
1376           ЛК Роделен БО 002P-01  RU000A107076    TQCB   96.74   1000.00   
2237               МОНОПОЛИЯ 001P-03  RU000A10ARS4    TQCB  101.25   1000.00   
1551               ГК Самолет БО-П13  RU000A107RZ0    TQCB   96.20   1000.00   
718             Лизинг-Трейд 001P-03  RU000A1034X1    TQCB   95.65    750.10   
1391              АРЕНЗА-ПРО 001P-03  RU000A107217    TQCB   98.38   1000.00   
710             ТЕХНО Лизинг 001P-04  RU000A1032X5    TQCB   93.70    900.00   
1183                   АБЗ-1 001P-04  RU000A1065B1    TQCB   96.29    505.00   
1130           Эталон-Финанс 002Р-01  RU000A105VU7    TQCB   95.72   1000.00   
785          Эталон-Финанс АО БО-П03  RU000A103QH9    TQCB   92.73    560.00   
2366        Совкомбанк Лизинг БО-П09  RU000A10B3W0    TQCB   99.96    874.90   
1165       Хайтэк-Интеграция 001P-01  RU000A106151    TQCB   96.71   1000.00   
1150           Страна Девелопмент 02  RU000A105YF2    TQCB   96.36   1000.00   
1560              АРЕНЗА-ПРО 001P-04  RU000A107ST1    TQCB   97.36   1000.00   
827               МСБ-Лизинг 002P-05  RU000A103VD8    TQCB   95.17    388.84   
1232              МСБ-Лизинг 003P-01  RU000A106D18    TQCB   96.81    416.62   
1255              АРЕНЗА-ПРО 001P-02  RU000A106GC4    TQCB   95.75    600.00   
872             ГК Пионер БО 001P-06  RU000A104735    TQCB   98.34    500.00   
1319             Интерлизинг 001Р-06  RU000A106SF2    TQCB   96.53    835.00   
1366               Джи-групп 002P-03  RU000A106Z38    TQCB   97.07    750.00   
1059          АФК Система БО 001Р-24  RU000A105L27    TQCB   96.74   1000.00   
368                АЗБУКА_ВКУСА_БОП1  RU000A100HE1    TQCB   97.50   1000.00   
1089            ЕвроТранс БО-001Р-01  RU000A105PP9    TQCB   97.90   1000.00   
2051        Сбербанк ПАО 002Р-SBER48  RU000A10A3L4    TQCB   99.92   1000.00   
1013             ПАТРИОТ ГРУПП БО-01  RU000A105AX5    TQCB   98.96   1000.00   
1117            ЕвроТранс БО-001Р-02  RU000A105TS5    TQCB   97.22   1000.00   
1276   Группа компаний Медси 001P-03  RU000A106K27    TQCB   98.27   1000.00   
838           Банк ВТБ (ПАО) Б-1-256  RU000A103XU8    TQCB   90.58   1000.00   
1643     "Сэтл-Групп" ООО БО 002P-03  RU000A1084B2    TQCB   97.80   1000.00   
1210                Софтлайн 002Р-01  RU000A106A78    TQCB   96.44   1000.00   
871           АФК Система БО 001P-23  RU000A104693    TQCB   94.97   1000.00   
395           АФК Система БО 001P-12  RU000A101012    TQCB  100.12   1000.00   
337           АФК Система БО 001P-09  RU000A1005L6    TQCB   95.69   1000.00   
877   Первое клиент.бюро НАО 001Р-03  RU000A1047S3    TQCB   98.65    200.00   
1116              АРЕНЗА-ПРО 001P-01  RU000A105TR7    TQCB   98.35    400.00   
1191            Южуралзолото 001P-03  RU000A106656    TQCB   97.79   1000.00   
1174       Аквилон-Лизинг БО-02-001P  RU000A1063Y8    TQCB   98.20    750.00   
1622                 ГЛОРАКС 001Р-02  RU000A108132    TQCB   99.43   1000.00   
783        Группа ЛСР ПАО БО 001Р-07  RU000A103PX8    TQCB   94.74    800.00   
349           АФК Система БО 001P-10  RU000A1008J4    TQCB   98.70   1000.00   
879            РЕСО-Лизинг ООО БО-06  RU000A1049Y7    TQCB  100.40   1000.00   
1094      Уральская Сталь БО-001Р-01  RU000A105Q63    TQCB   96.74   1000.00   
1113                   АБЗ-1 001P-03  RU000A105SX7    TQCB   98.59    340.00   
1828        Сбербанк ПАО 002Р-SBER47  RU000A1092X9    TQCB  100.09   1000.00   
1628          Новые технологии БO-03  RU000A1082W2    TQCB   98.75   1000.00   
443           АФК Система БО 001P-13  RU000A101Q26    TQCB   99.41   1000.00   
670        Группа ЛСР ПАО БО 

In [ ]:
bonds_best[bonds_best['RATING'] == '']

In [18]:
# расчет среднего значения доходности в рейтинге
bonds_best[bonds_best['RATING'] == 'A-'][['YIELD']].mean()

YIELD    32.098889
dtype: float64

In [ ]:
# таблица для моих облигаций
create_my_bonds_table = """
CREATE TABLE IF NOT EXISTS my_bonds (
  isin TEXT NOT NULL PRIMARY KEY,
  name TEXT NOT NULL,
  count INTEGER NOT NULL
);
"""

In [ ]:
execute_query(connection, create_my_bonds_table)

In [39]:
read_my_bonds_data = """
SELECT * FROM my_bonds;
"""

In [81]:
# Чтение данных из базы для получения списка моих бондов
my_bonds = pd.read_sql(read_my_bonds_data, connection)
my_bonds = my_bonds.rename(columns = {'isin':'SECID','name':'SECNAME','count':'COUNT'})
# Обогащение данными с Мосбиржи
my_bonds = pd.merge(my_bonds[['SECID','COUNT']], bonds, on = 'SECID', how = 'left')
my_bonds

SECID  COUNT                        SECNAME BOARDID   LAST  \
0   RU000A106WZ2   13.0                АПРИ БО-002Р-03    TQCB  98.52   
1   RU000A1008J4    9.0         АФК Система БО 001P-10    TQCB  98.70   
2   RU000A107UU5   10.0               Брусника 002Р-02    TQCB  98.61   
3   RU000A107RZ0   21.0              ГК Самолет БО-П13    TQCB  96.20   
4   RU000A105PP9    8.0           ЕвроТранс БО-001Р-01    TQCB  97.90   
5   RU000A105CS1   10.0               Селигдар 001Р-01    TQCB  98.10   
6   RU000A106K27   12.0  Группа компаний Медси 001P-03    TQCB  98.27   
7   RU000A100HE1    4.0              АЗБУКА_ВКУСА_БОП1    TQCB  97.50   
8   RU000A1065B1   12.0                  АБЗ-1 001P-04    TQCB  96.29   
9   RU000A106D18   13.0             МСБ-Лизинг 003P-01    TQCB  96.81   
10  RU000A106151    5.0      Хайтэк-Интеграция 001P-01    TQCB  96.71   
11  RU000A105VU7    5.0          Эталон-Финанс 002Р-01    TQCB  95.72   

    LOTVALUE  YIELD  DURATION   OFFERDATE     MATDATE  COUPONPERCENT  \
0    1000.00  39.44        52  2025-09-30  2027-09-14          24.00   
1    1000.00  19.84        54  2025-10-01  2029-03-23           9.90   
2    1000.00  33.12        41  2025-09-18  2027-03-28          16.25   
3    1000.00  25.61       173  2026-02-03  2027-01-24          14.75   
4    1000.00  21.82       122        None  2025-12-11          13.50   
5    1000.00  19.05        82        None  2025-10-29          10.80   
6    1000.00  21.59        68  2025-10-15  2038-06-28          10.40   
7    1000.00  21.85        47  2025-09-24  2029-06-15           0.10   
8     505.00  24.56       161        None  2026-04-21          13.75   
9     416.62  23.61       151        None  2026-05-28          13.50   
10   1000.00  23.91       177        None  2026-03-26          15.00   
11   1000.00  24.31       187  2026-02-20  2038-02-03          13.70   

    ACCRUEDINT  VOLTODAY RATING Qual  
0        34.19       207   BBB-  Нет  
1        36.07       141    AA-  Нет  
2        10.24      1641     A-  Нет  
3         2.42      6286      A  Нет  
4         9.25       578     A-  Нет  
5        29.59       518     A+  Нет  
6         7.12        64    AA-  Нет  
7         0.13       315     A-  Нет  
8         3.23       276   BBB+  Нет  
9         1.08         1   BBB-  Нет  
10       17.67        89   BBB+  Нет  
11       29.65       871     A-  Нет

In [40]:
# Удаляю проданные бумаги
my_bonds = my_bonds.drop([11])
my_bonds

SECID  COUNT                        SECNAME BOARDID   LAST  \
0   RU000A106WZ2   13.0                АПРИ БО-002Р-03    TQCB  98.36   
1   RU000A1008J4    9.0         АФК Система БО 001P-10    TQCB  98.48   
2   RU000A107UU5   10.0               Брусника 002Р-02    TQCB  98.50   
3   RU000A107RZ0   21.0              ГК Самолет БО-П13    TQCB  96.09   
4   RU000A105PP9    8.0           ЕвроТранс БО-001Р-01    TQCB  98.30   
5   RU000A105CS1   10.0               Селигдар 001Р-01    TQCB  98.24   
6   RU000A106K27   12.0  Группа компаний Медси 001P-03    TQCB  98.80   
7   RU000A100HE1    4.0              АЗБУКА_ВКУСА_БОП1    TQCB  97.55   
8   RU000A1065B1   12.0                  АБЗ-1 001P-04    TQCB  96.38   
9   RU000A106D18   13.0             МСБ-Лизинг 003P-01    TQCB  96.59   
10  RU000A106151    5.0      Хайтэк-Интеграция 001P-01    TQCB  96.51   

    LOTVALUE  YIELD  DURATION   OFFERDATE     MATDATE  COUPONPERCENT  \
0    1000.00  40.42        54  2025-09-30  2027-09-14          24.00   
1    1000.00  21.18        56  2025-10-01  2029-03-23           9.90   
2    1000.00  33.61        43  2025-09-18  2027-03-28          16.25   
3    1000.00  25.80       175  2026-02-03  2027-01-24          14.75   
4    1000.00  20.25       124        None  2025-12-11          13.50   
5    1000.00  18.24        84        None  2025-10-29          10.80   
6    1000.00  17.94        70  2025-10-15  2038-06-28          10.40   
7    1000.00  20.41        49  2025-09-24  2029-06-15           0.10   
8     505.00  24.18       163        None  2026-04-21          13.75   
9     416.62  24.16       153        None  2026-05-28          13.50   
10   1000.00  24.34       179        None  2026-03-26          15.00   

    ACCRUEDINT  VOLTODAY RATING Qual  
0        32.88       330   BBB-  Нет  
1        35.53       207    AA-  Нет  
2         9.35      5353     A-  Нет  
3         1.62     20734      A  Нет  
4         8.51       619     A-  Нет  
5        29.00       346     A+  Нет  
6         6.55        94    AA-  Нет  
7         0.13        59     A-  Нет  
8         2.85       666   BBB+  Нет  
9         0.77       433   BBB-  Нет  
10       16.85       771   BBB+  Нет

In [41]:
# Заполняю датафрейм с моими бондами, добавляю новые в конец датафрейма
my_bonds = pd.concat([my_bonds,bonds[bonds['SECID'] == 'RU000A105VU7']])
# указываю количетсво бумаг в портфеле
my_bonds.loc[my_bonds['SECID'] == 'RU000A105VU7', 'COUNT'] = 5
# заполняю цены там, где не хватает
# my_bonds.loc[my_bonds['SECID'] == 'RU000A106EM8', 'LAST'] = 101.45
my_bonds

SECID  COUNT                        SECNAME BOARDID   LAST  \
0     RU000A106WZ2   13.0                АПРИ БО-002Р-03    TQCB  98.36   
1     RU000A1008J4    9.0         АФК Система БО 001P-10    TQCB  98.48   
2     RU000A107UU5   10.0               Брусника 002Р-02    TQCB  98.50   
3     RU000A107RZ0   21.0              ГК Самолет БО-П13    TQCB  96.09   
4     RU000A105PP9    8.0           ЕвроТранс БО-001Р-01    TQCB  98.30   
5     RU000A105CS1   10.0               Селигдар 001Р-01    TQCB  98.24   
6     RU000A106K27   12.0  Группа компаний Медси 001P-03    TQCB  98.80   
7     RU000A100HE1    4.0              АЗБУКА_ВКУСА_БОП1    TQCB  97.55   
8     RU000A1065B1   12.0                  АБЗ-1 001P-04    TQCB  96.38   
9     RU000A106D18   13.0             МСБ-Лизинг 003P-01    TQCB  96.59   
10    RU000A106151    5.0      Хайтэк-Интеграция 001P-01    TQCB  96.51   
1127  RU000A105VU7    5.0          Эталон-Финанс 002Р-01    TQCB  95.58   

      LOTVALUE  YIELD  DURATION   OFFERDATE     MATDATE  COUPONPERCENT  \
0      1000.00  40.42        54  2025-09-30  2027-09-14          24.00   
1      1000.00  21.18        56  2025-10-01  2029-03-23           9.90   
2      1000.00  33.61        43  2025-09-18  2027-03-28          16.25   
3      1000.00  25.80       175  2026-02-03  2027-01-24          14.75   
4      1000.00  20.25       124        None  2025-12-11          13.50   
5      1000.00  18.24        84        None  2025-10-29          10.80   
6      1000.00  17.94        70  2025-10-15  2038-06-28          10.40   
7      1000.00  20.41        49  2025-09-24  2029-06-15           0.10   
8       505.00  24.18       163        None  2026-04-21          13.75   
9       416.62  24.16       153        None  2026-05-28          13.50   
10     1000.00  24.34       179        None  2026-03-26          15.00   
1127   1000.00  24.55       189  2026-02-20  2038-02-03          13.70   

      ACCRUEDINT  VOLTODAY RATING Qual  
0          32.88       330   BBB-  Нет  
1          35.53       207    AA-  Нет  
2           9.35      5353     A-  Нет  
3           1.62     20734      A  Нет  
4           8.51       619     A-  Нет  
5          29.00       346     A+  Нет  
6           6.55        94    AA-  Нет  
7           0.13        59     A-  Нет  
8           2.85       666   BBB+  Нет  
9           0.77       433   BBB-  Нет  
10         16.85       771   BBB+  Нет  
1127       28.90       874     A-  Нет

In [82]:
# расчет стоимости бондов в портфеле
my_bonds['VALUE'] = None
for index, row in my_bonds.iterrows():
    my_bonds.loc[index, 'VALUE'] = my_bonds.loc[index]['LAST'] * my_bonds.loc[index]['LOTVALUE'] * my_bonds.loc[index]['COUNT'] / 100

# определяю долю бумаги в портфеле
value_sum = my_bonds['VALUE'].sum()
for index, row in my_bonds.iterrows():
    my_bonds.loc[index, 'PART'] = (my_bonds.loc[index, 'VALUE']*100 / value_sum).round(2)
    if (my_bonds.loc[index, 'PART'] > 8):
        my_bonds.loc[index, 'SELLCOUNT'] = math.ceil((value_sum*7/100 - my_bonds.loc[index, 'VALUE']) / (my_bonds.loc[index, 'LAST']*my_bonds.loc[index, 'LOTVALUE']/100))
    else:
        my_bonds.loc[index, 'SELLCOUNT'] = 0

my_bonds = my_bonds.drop(['BOARDID','Qual'], axis=1)
my_bonds

SECID  COUNT                        SECNAME   LAST  LOTVALUE  \
0   RU000A106WZ2   13.0                АПРИ БО-002Р-03  98.52   1000.00   
1   RU000A1008J4    9.0         АФК Система БО 001P-10  98.70   1000.00   
2   RU000A107UU5   10.0               Брусника 002Р-02  98.61   1000.00   
3   RU000A107RZ0   21.0              ГК Самолет БО-П13  96.20   1000.00   
4   RU000A105PP9    8.0           ЕвроТранс БО-001Р-01  97.90   1000.00   
5   RU000A105CS1   10.0               Селигдар 001Р-01  98.10   1000.00   
6   RU000A106K27   12.0  Группа компаний Медси 001P-03  98.27   1000.00   
7   RU000A100HE1    4.0              АЗБУКА_ВКУСА_БОП1  97.50   1000.00   
8   RU000A1065B1   12.0                  АБЗ-1 001P-04  96.29    505.00   
9   RU000A106D18   13.0             МСБ-Лизинг 003P-01  96.81    416.62   
10  RU000A106151    5.0      Хайтэк-Интеграция 001P-01  96.71   1000.00   
11  RU000A105VU7    5.0          Эталон-Финанс 002Р-01  95.72   1000.00   

    YIELD  DURATION   OFFERDATE     MATDATE  COUPONPERCENT  ACCRUEDINT  \
0   39.44        52  2025-09-30  2027-09-14          24.00       34.19   
1   19.84        54  2025-10-01  2029-03-23           9.90       36.07   
2   33.12        41  2025-09-18  2027-03-28          16.25       10.24   
3   25.61       173  2026-02-03  2027-01-24          14.75        2.42   
4   21.82       122        None  2025-12-11          13.50        9.25   
5   19.05        82        None  2025-10-29          10.80       29.59   
6   21.59        68  2025-10-15  2038-06-28          10.40        7.12   
7   21.85        47  2025-09-24  2029-06-15           0.10        0.13   
8   24.56       161        None  2026-04-21          13.75        3.23   
9   23.61       151        None  2026-05-28          13.50        1.08   
10  23.91       177        None  2026-03-26          15.00       17.67   
11  24.31       187  2026-02-20  2038-02-03          13.70       29.65   

    VOLTODAY RATING        VALUE   PART  SELLCOUNT  
0        207   BBB-      12807.6  12.11       -5.0  
1        141    AA-       8883.0   8.40       -1.0  
2       1641     A-       9861.0   9.32       -2.0  
3       6286      A      20202.0  19.10      -13.0  
4        578     A-       7832.0   7.40        0.0  
5        518     A+       9810.0   9.27       -2.0  
6         64    AA-      11792.4  11.15       -4.0  
7        315     A-       3900.0   3.69        0.0  
8        276   BBB+     5835.174   5.52        0.0  
9          1   BBB-  5243.287686   4.96        0.0  
10        89   BBB+       4835.5   4.57        0.0  
11       871     A-       4786.0   4.52        0.0

In [50]:
# функция для поиска изменений в рейтинге
def change_rating(bonds_list):
    try:
        change_rating_bonds = pd.DataFrame() # пустой датафрейм
        for index, row in bonds_list.iterrows():
            isin = bonds_list.loc[index, 'SECID']
            url = "https://smart-lab.ru/q/bonds/" + isin # формироние url
            response = requests.get(url) # запрос
            rating = get_creadit_rating(response) # присваивает полученное значение рейтинга очередному элементу
            if (rating != bonds_list.loc[index, 'RATING']):
                change_rating_bonds = pd.concat([change_rating_bonds,bonds_list[bonds_list['SECID'] == isin]])
            else:
                continue
    except Exception as e:
        print(f"Ошибка получении рейтингов: {e}" + isin) # если не удалось выполнить запрос для какого-то isin (ошибка выполнения запроса)

    return change_rating_bonds

In [51]:
change_rating(my_bonds)

Empty DataFrame
Columns: []
Index: []

In [83]:
value_sum

np.float64(105787.961686)

In [84]:
my_bonds['YIELD'].mean()

np.float64(24.892500000000002)

In [47]:
# обновление таблицы с моими облигациями в БД
bonds_for_db = my_bonds[['SECID','SECNAME','COUNT']].rename(columns = {'SECID':'isin','SECNAME':'name','COUNT':'count'})
bonds_for_db.to_sql(name = 'my_bonds', con = connection, if_exists = 'replace', index = False) # также с заменой значений, чтобы не отслеживать те, что были исключены из портфеля

12

In [ ]:
# таблица для портфеля из облигаций
create_my_bonds_portfolio_table = """
CREATE TABLE IF NOT EXISTS my_bonds_portfolio (
  isin TEXT NOT NULL PRIMARY KEY,
  name TEXT NOT NULL,
  count INTEGER NOT NULL
);
"""

In [ ]:
execute_query(connection, create_my_bonds_portfolio_table)

In [42]:
read_my_bonds_portfolio_data = """
SELECT * FROM my_bonds_portfolio;
"""

In [85]:
# Чтение данных из базы для получения списка моих бондов
my_bonds_portfolio = pd.read_sql(read_my_bonds_portfolio_data, connection)
my_bonds_portfolio = my_bonds_portfolio.rename(columns = {'isin':'SECID','name':'SECNAME','count':'COUNT'})
# Обогащение данными с Мосбиржи
my_bonds_portfolio = pd.merge(my_bonds_portfolio[['SECID','COUNT']], bonds, on = 'SECID', how = 'left')
my_bonds_portfolio

SECID  COUNT                    SECNAME BOARDID    LAST  LOTVALUE  \
0   RU000A107217   12.0         АРЕНЗА-ПРО 001P-03    TQCB   98.38    1000.0   
1   RU000A106656    6.0       Южуралзолото 001P-03    TQCB   97.79    1000.0   
2   RU000A106WZ2    5.0            АПРИ БО-002Р-03    TQCB   98.52    1000.0   
3   RU000A1008J4    5.0     АФК Система БО 001P-10    TQCB   98.70    1000.0   
4   RU000A107076    8.0      ЛК Роделен БО 002P-01    TQCB   96.74    1000.0   
5   RU000A10ARS4    6.0          МОНОПОЛИЯ 001P-03    TQCB  101.25    1000.0   
6   RU000A106151    7.0  Хайтэк-Интеграция 001P-01    TQCB   96.71    1000.0   
7   RU000A105Y14    4.0      КЛВЗ КРИСТАЛЛ 001P-01    TQCB   98.76     750.0   
8   RU000A105SX7    5.0              АБЗ-1 001P-03    TQCB   98.59     340.0   
9   RU000A104735   11.0       ГК Пионер БО 001P-06    TQCB   98.34     500.0   
10  RU000A105PP9   10.0       ЕвроТранс БО-001Р-01    TQCB   97.90    1000.0   
11  RU000A1034X1   10.0       Лизинг-Трейд 001P-03    TQCB   95.65     750.1   
12  RU000A105VU7    7.0      Эталон-Финанс 002Р-01    TQCB   95.72    1000.0   
13  RU000A1065B1   14.0              АБЗ-1 001P-04    TQCB   96.29     505.0   

    YIELD  DURATION   OFFERDATE     MATDATE  COUPONPERCENT  ACCRUEDINT  \
0   24.73        89  2025-11-06  2028-11-21          15.50        3.40   
1   20.52        76        None  2025-10-23          10.05       29.19   
2   39.44        52  2025-09-30  2027-09-14          24.00       34.19   
3   19.84        54  2025-10-01  2029-03-23           9.90       36.07   
4   26.59       152  2026-01-12  2028-09-13          15.75        3.45   
5   26.37       161        None  2026-01-25          26.50        7.26   
6   23.91       177        None  2026-03-26          15.00       17.67   
7   18.51       118        None  2026-03-10          13.50       16.37   
8   19.75       128        None  2026-01-29          14.25        1.06   
9   22.11        71        None  2025-12-04          11.75       10.30   
10  21.82       122        None  2025-12-11          13.50        9.25   
11  25.21       140        None  2026-04-30          11.00        1.13   
12  24.31       187  2026-02-20  2038-02-03          13.70       29.65   
13  24.56       161        None  2026-04-21          13.75        3.23   

    VOLTODAY RATING Qual  
0          7    BBB  Нет  
1       1881     AA  Нет  
2        207   BBB-  Нет  
3        141    AA-  Нет  
4         32   BBB+  Нет  
5       9997   BBB+  Нет  
6         89   BBB+  Нет  
7         47    BB-  Нет  
8        268   BBB+  Нет  
9        590     A-  Нет  
10       578     A-  Нет  
11        42   BBB-  Нет  
12       871     A-  Нет  
13       276   BBB+  Нет

In [66]:
# Заполняю датафрейм с моими бондами, добавляю новые в конец датафрейма
# my_bonds_portfolio = pd.DataFrame() # пустой датафрейм
my_bonds_portfolio = pd.concat([my_bonds_portfolio,bonds[bonds['SECID'] == 'RU000A1065B1']])
# указываю количетсво бумаг в портфеле
my_bonds_portfolio.loc[my_bonds_portfolio['SECID'] == 'RU000A1065B1', 'COUNT'] = 14
# заполняю цены там, где не хватает
# my_bonds.loc[my_bonds['SECID'] == 'RU000A106EM8', 'LAST'] = 101.45
my_bonds_portfolio

SECID  COUNT                    SECNAME    LAST  LOTVALUE  YIELD  \
0     RU000A107217   12.0         АРЕНЗА-ПРО 001P-03   98.41    1000.0  24.57   
2     RU000A106656    6.0       Южуралзолото 001P-03   97.80    1000.0  20.47   
3     RU000A106WZ2    5.0            АПРИ БО-002Р-03   98.44    1000.0  40.21   
4     RU000A1008J4    5.0     АФК Система БО 001P-10   98.48    1000.0  21.61   
5     RU000A107076    8.0      ЛК Роделен БО 002P-01   96.74    1000.0  26.59   
6     RU000A10ARS4    6.0          МОНОПОЛИЯ 001P-03  101.29    1000.0  26.25   
7     RU000A106151    7.0  Хайтэк-Интеграция 001P-01   96.50    1000.0  24.46   
8     RU000A105Y14    4.0      КЛВЗ КРИСТАЛЛ 001P-01   98.76     750.0  18.51   
9     RU000A105SX7    5.0              АБЗ-1 001P-03   98.58     340.0  19.79   
10    RU000A104735   11.0       ГК Пионер БО 001P-06   98.37     500.0  21.92   
12    RU000A105PP9   10.0       ЕвроТранс БО-001Р-01   97.96    1000.0  21.59   
14    RU000A1034X1   10.0       Лизинг-Трейд 001P-03   95.65     750.1  25.21   
1130  RU000A105VU7    7.0      Эталон-Финанс 002Р-01   95.71    1000.0  24.34   
1183  RU000A1065B1   14.0              АБЗ-1 001P-04   96.28     505.0  24.59   

      DURATION   OFFERDATE     MATDATE  COUPONPERCENT  ACCRUEDINT  VOLTODAY  \
0           89  2025-11-06  2028-11-21          15.50        3.40         3   
2           76        None  2025-10-23          10.05       29.19      1529   
3           52  2025-09-30  2027-09-14          24.00       34.19       145   
4           54  2025-10-01  2029-03-23           9.90       36.07        69   
5          152  2026-01-12  2028-09-13          15.75        3.45        32   
6          161        None  2026-01-25          26.50        7.26      9784   
7          177        None  2026-03-26          15.00       17.67        88   
8          118        None  2026-03-10          13.50       16.37        47   
9          128        None  2026-01-29          14.25        1.06       159   
10          71        None  2025-12-04          11.75       10.30       511   
12         122        None  2025-12-11          13.50        9.25       431   
14         140        None  2026-04-30          11.00        1.13         6   
1130       187  2026-02-20  2038-02-03          13.70       29.65       354   
1183       161        None  2026-04-21          13.75        3.23       247   

     RATING      VALUE   PART  SELLCOUNT BOARDID Qual  
0       BBB    11809.2  13.94       -5.0     NaN  NaN  
2        AA     5868.0   6.93        0.0     NaN  NaN  
3      BBB-     4922.0   5.81        0.0     NaN  NaN  
4       AA-     4924.0   5.81        0.0     NaN  NaN  
5      BBB+     7739.2   9.13       -1.0     NaN  NaN  
6      BBB+     6077.4   7.17        0.0     NaN  NaN  
7      BBB+     6755.0   7.97        0.0     NaN  NaN  
8       BB-     2962.8   3.50        0.0     NaN  NaN  
9      BBB+    1675.86   1.98        0.0     NaN  NaN  
10       A-    5410.35   6.39        0.0     NaN  NaN  
12       A-     9796.0  11.56       -3.0     NaN  NaN  
14     BBB-  7174.7065   8.47       -1.0     NaN  NaN  
1130     A-     6699.7   7.91        0.0     NaN  NaN  
1183   BBB+        NaN    NaN        NaN    TQCB  Нет

In [65]:
# Удаляю проданные
my_bonds_portfolio = my_bonds_portfolio.drop([11])
my_bonds_portfolio

SECID  COUNT                    SECNAME    LAST  LOTVALUE  YIELD  \
0     RU000A107217   12.0         АРЕНЗА-ПРО 001P-03   98.41    1000.0  24.57   
2     RU000A106656    6.0       Южуралзолото 001P-03   97.80    1000.0  20.47   
3     RU000A106WZ2    5.0            АПРИ БО-002Р-03   98.44    1000.0  40.21   
4     RU000A1008J4    5.0     АФК Система БО 001P-10   98.48    1000.0  21.61   
5     RU000A107076    8.0      ЛК Роделен БО 002P-01   96.74    1000.0  26.59   
6     RU000A10ARS4    6.0          МОНОПОЛИЯ 001P-03  101.29    1000.0  26.25   
7     RU000A106151    7.0  Хайтэк-Интеграция 001P-01   96.50    1000.0  24.46   
8     RU000A105Y14    4.0      КЛВЗ КРИСТАЛЛ 001P-01   98.76     750.0  18.51   
9     RU000A105SX7    5.0              АБЗ-1 001P-03   98.58     340.0  19.79   
10    RU000A104735   11.0       ГК Пионер БО 001P-06   98.37     500.0  21.92   
12    RU000A105PP9   10.0       ЕвроТранс БО-001Р-01   97.96    1000.0  21.59   
14    RU000A1034X1   10.0       Лизинг-Трейд 001P-03   95.65     750.1  25.21   
1130  RU000A105VU7    7.0      Эталон-Финанс 002Р-01   95.71    1000.0  24.34   

      DURATION   OFFERDATE     MATDATE  COUPONPERCENT  ACCRUEDINT  VOLTODAY  \
0           89  2025-11-06  2028-11-21          15.50        3.40         3   
2           76        None  2025-10-23          10.05       29.19      1529   
3           52  2025-09-30  2027-09-14          24.00       34.19       145   
4           54  2025-10-01  2029-03-23           9.90       36.07        69   
5          152  2026-01-12  2028-09-13          15.75        3.45        32   
6          161        None  2026-01-25          26.50        7.26      9784   
7          177        None  2026-03-26          15.00       17.67        88   
8          118        None  2026-03-10          13.50       16.37        47   
9          128        None  2026-01-29          14.25        1.06       159   
10          71        None  2025-12-04          11.75       10.30       511   
12         122        None  2025-12-11          13.50        9.25       431   
14         140        None  2026-04-30          11.00        1.13         6   
1130       187  2026-02-20  2038-02-03          13.70       29.65       354   

     RATING      VALUE   PART  SELLCOUNT  
0       BBB    11809.2  13.94       -5.0  
2        AA     5868.0   6.93        0.0  
3      BBB-     4922.0   5.81        0.0  
4       AA-     4924.0   5.81        0.0  
5      BBB+     7739.2   9.13       -1.0  
6      BBB+     6077.4   7.17        0.0  
7      BBB+     6755.0   7.97        0.0  
8       BB-     2962.8   3.50        0.0  
9      BBB+    1675.86   1.98        0.0  
10       A-    5410.35   6.39        0.0  
12       A-     9796.0  11.56       -3.0  
14     BBB-  7174.7065   8.47       -1.0  
1130     A-     6699.7   7.91        0.0

In [67]:
# обновление таблицы с моими облигациями в БД
bonds_for_db = my_bonds_portfolio[['SECID','SECNAME','COUNT']].rename(columns = {'SECID':'isin','SECNAME':'name','COUNT':'count'})
bonds_for_db.to_sql(name = 'my_bonds_portfolio', con = connection, if_exists = 'replace', index = False) # также с заменой значений, чтобы не отслеживать те, что были исключены из портфеля

14

In [86]:
# расчет стоимости бондов в портфеле
my_bonds_portfolio['VALUE'] = None
for index, row in my_bonds_portfolio.iterrows():
    my_bonds_portfolio.loc[index, 'VALUE'] = my_bonds_portfolio.loc[index]['LAST'] * my_bonds_portfolio.loc[index]['LOTVALUE'] * my_bonds_portfolio.loc[index]['COUNT'] / 100

# определяю долю бумаги в портфеле
value_sum = my_bonds_portfolio['VALUE'].sum()
for index, row in my_bonds_portfolio.iterrows():
    my_bonds_portfolio.loc[index, 'PART'] = (my_bonds_portfolio.loc[index, 'VALUE']*100 / value_sum).round(2)
    if (my_bonds_portfolio.loc[index, 'PART'] > 8):
        my_bonds_portfolio.loc[index, 'SELLCOUNT'] = math.ceil((value_sum*7/100 - my_bonds_portfolio.loc[index, 'VALUE']) / (my_bonds_portfolio.loc[index, 'LAST']*my_bonds_portfolio.loc[index, 'LOTVALUE']/100))
    else:
        my_bonds_portfolio.loc[index, 'SELLCOUNT'] = 0

my_bonds_portfolio = my_bonds_portfolio.drop(['BOARDID','Qual'], axis=1)
my_bonds_portfolio

SECID  COUNT                    SECNAME    LAST  LOTVALUE  YIELD  \
0   RU000A107217   12.0         АРЕНЗА-ПРО 001P-03   98.38    1000.0  24.73   
1   RU000A106656    6.0       Южуралзолото 001P-03   97.79    1000.0  20.52   
2   RU000A106WZ2    5.0            АПРИ БО-002Р-03   98.52    1000.0  39.44   
3   RU000A1008J4    5.0     АФК Система БО 001P-10   98.70    1000.0  19.84   
4   RU000A107076    8.0      ЛК Роделен БО 002P-01   96.74    1000.0  26.59   
5   RU000A10ARS4    6.0          МОНОПОЛИЯ 001P-03  101.25    1000.0  26.37   
6   RU000A106151    7.0  Хайтэк-Интеграция 001P-01   96.71    1000.0  23.91   
7   RU000A105Y14    4.0      КЛВЗ КРИСТАЛЛ 001P-01   98.76     750.0  18.51   
8   RU000A105SX7    5.0              АБЗ-1 001P-03   98.59     340.0  19.75   
9   RU000A104735   11.0       ГК Пионер БО 001P-06   98.34     500.0  22.11   
10  RU000A105PP9   10.0       ЕвроТранс БО-001Р-01   97.90    1000.0  21.82   
11  RU000A1034X1   10.0       Лизинг-Трейд 001P-03   95.65     750.1  25.21   
12  RU000A105VU7    7.0      Эталон-Финанс 002Р-01   95.72    1000.0  24.31   
13  RU000A1065B1   14.0              АБЗ-1 001P-04   96.29     505.0  24.56   

    DURATION   OFFERDATE     MATDATE  COUPONPERCENT  ACCRUEDINT  VOLTODAY  \
0         89  2025-11-06  2028-11-21          15.50        3.40         7   
1         76        None  2025-10-23          10.05       29.19      1881   
2         52  2025-09-30  2027-09-14          24.00       34.19       207   
3         54  2025-10-01  2029-03-23           9.90       36.07       141   
4        152  2026-01-12  2028-09-13          15.75        3.45        32   
5        161        None  2026-01-25          26.50        7.26      9997   
6        177        None  2026-03-26          15.00       17.67        89   
7        118        None  2026-03-10          13.50       16.37        47   
8        128        None  2026-01-29          14.25        1.06       268   
9         71        None  2025-12-04          11.75       10.30       590   
10       122        None  2025-12-11          13.50        9.25       578   
11       140        None  2026-04-30          11.00        1.13        42   
12       187  2026-02-20  2038-02-03          13.70       29.65       871   
13       161        None  2026-04-21          13.75        3.23       276   

   RATING      VALUE   PART  SELLCOUNT  
0     BBB    11805.6  13.32       -5.0  
1      AA     5867.4   6.62        0.0  
2    BBB-     4926.0   5.56        0.0  
3     AA-     4935.0   5.57        0.0  
4    BBB+     7739.2   8.73       -1.0  
5    BBB+     6075.0   6.85        0.0  
6    BBB+     6769.7   7.64        0.0  
7     BB-     2962.8   3.34        0.0  
8    BBB+    1676.03   1.89        0.0  
9      A-     5408.7   6.10        0.0  
10     A-     9790.0  11.04       -3.0  
11   BBB-  7174.7065   8.09       -1.0  
12     A-     6700.4   7.56        0.0  
13   BBB+   6807.703   7.68        0.0

In [61]:
change_rating(my_bonds_portfolio)

Empty DataFrame
Columns: []
Index: []

In [87]:
value_sum

np.float64(88638.23949999998)

In [88]:
my_bonds_portfolio['YIELD'].mean()

np.float64(24.119285714285713)

In [89]:
my_bonds_portfolio['COUNT'].sum()

np.float64(110.0)

In [62]:
# таблица для портфеля из облигаций в Finam
create_my_bonds_finam = """
CREATE TABLE IF NOT EXISTS my_bonds_finam (
  isin TEXT NOT NULL PRIMARY KEY,
  name TEXT NOT NULL,
  count INTEGER NOT NULL
);
"""

In [91]:
# Заполняю датафрейм с моими бондами, добавляю новые в конец датафрейма
# my_bonds_finam = pd.DataFrame() # пустой датафрейм
my_bonds_finam = pd.concat([my_bonds_finam,bonds[bonds['SECID'] == 'RU000A105VU7']])
# указываю количетсво бумаг в портфеле
my_bonds_finam.loc[my_bonds_finam['SECID'] == 'RU000A105VU7', 'COUNT'] = 9
# заполняю цены там, где не хватает
# my_bonds.loc[my_bonds['SECID'] == 'RU000A106EM8', 'LAST'] = 101.45
my_bonds_finam

SECID  COUNT                SECNAME    LAST  LOTVALUE  YIELD  \
0     RU000A10C8H9    1.0     Оил Ресурс 001P-02  100.08    1000.0  31.85   
1130  RU000A105VU7    9.0  Эталон-Финанс 002Р-01   95.72    1000.0  24.31   

      DURATION   OFFERDATE     MATDATE  COUPONPERCENT  ACCRUEDINT  VOLTODAY  \
0          987        None  2030-07-06           28.0        5.37      1978   
1130       187  2026-02-20  2038-02-03           13.7       29.65       871   

     RATING   VALUE   PART  SELLCOUNT BOARDID Qual  
0         -  1000.8  11.56        0.0     NaN  NaN  
1130     A-     NaN    NaN        NaN    TQCB  Нет

In [92]:
# обновление таблицы с моими облигациями в БД
bonds_for_db = my_bonds_finam[['SECID','SECNAME','COUNT']].rename(columns = {'SECID':'isin','SECNAME':'name','COUNT':'count'})
bonds_for_db.to_sql(name = 'my_bonds_finam', con = connection, if_exists = 'replace', index = False) # также с заменой значений, чтобы не отслеживать те, что были исключены из портфеля

2

In [46]:
read_my_bonds_finam_data = """
SELECT * FROM my_bonds_finam;
"""

In [90]:
# Удаляю проданные
my_bonds_finam = my_bonds_finam.drop([1])
my_bonds_finam

SECID  COUNT             SECNAME    LAST  LOTVALUE  YIELD  DURATION  \
0  RU000A10C8H9    1.0  Оил Ресурс 001P-02  100.08    1000.0  31.85       987   

  OFFERDATE     MATDATE  COUPONPERCENT  ACCRUEDINT  VOLTODAY RATING   VALUE  \
0      None  2030-07-06           28.0        5.37      1978      -  1000.8   

    PART  SELLCOUNT  
0  11.56        0.0

In [47]:
# Чтение данных из базы для получения списка моих бондов
my_bonds_finam = pd.read_sql(read_my_bonds_finam_data, connection)
my_bonds_finam = my_bonds_finam.rename(columns = {'isin':'SECID','name':'SECNAME','count':'COUNT'})
# Обогащение данными с Мосбиржи
my_bonds_finam = pd.merge(my_bonds_finam[['SECID','COUNT']], bonds, on = 'SECID', how = 'left')
my_bonds_finam

SECID  COUNT                SECNAME BOARDID    LAST  LOTVALUE  \
0  RU000A10C8H9    1.0     Оил Ресурс 001P-02    TQCB  100.08    1000.0   
1  RU000A105VU7    8.0  Эталон-Финанс 002Р-01    TQCB   95.71    1000.0   

   YIELD  DURATION   OFFERDATE     MATDATE  COUPONPERCENT  ACCRUEDINT  \
0  31.85       987        None  2030-07-06           28.0        5.37   
1  24.34       187  2026-02-20  2038-02-03           13.7       29.65   

   VOLTODAY RATING Qual  
0      1978      -  Нет  
1       354     A-  Нет

In [93]:
# расчет стоимости бондов в портфеле
my_bonds_finam['VALUE'] = None
for index, row in my_bonds_finam.iterrows():
    my_bonds_finam.loc[index, 'VALUE'] = my_bonds_finam.loc[index]['LAST'] * my_bonds_finam.loc[index]['LOTVALUE'] * my_bonds_finam.loc[index]['COUNT'] / 100

# определяю долю бумаги в портфеле
value_sum = my_bonds_finam['VALUE'].sum()
for index, row in my_bonds_finam.iterrows():
    my_bonds_finam.loc[index, 'PART'] = (my_bonds_finam.loc[index, 'VALUE']*100 / value_sum).round(2)
    if (my_bonds_finam.loc[index, 'PART'] > 8):
        my_bonds_finam.loc[index, 'SELLCOUNT'] = math.ceil((value_sum*7/100 - my_bonds_finam.loc[index, 'VALUE']) / (my_bonds_finam.loc[index, 'LAST']*my_bonds_finam.loc[index, 'LOTVALUE']/100))
    else:
        my_bonds_finam.loc[index, 'SELLCOUNT'] = 0

my_bonds_finam = my_bonds_finam.drop(['BOARDID','Qual'], axis=1)
my_bonds_finam

SECID  COUNT                SECNAME    LAST  LOTVALUE  YIELD  \
0     RU000A10C8H9    1.0     Оил Ресурс 001P-02  100.08    1000.0  31.85   
1130  RU000A105VU7    9.0  Эталон-Финанс 002Р-01   95.72    1000.0  24.31   

      DURATION   OFFERDATE     MATDATE  COUPONPERCENT  ACCRUEDINT  VOLTODAY  \
0          987        None  2030-07-06           28.0        5.37      1978   
1130       187  2026-02-20  2038-02-03           13.7       29.65       871   

     RATING   VALUE   PART  SELLCOUNT  
0         -  1000.8  10.41        0.0  
1130     A-  8614.8  89.59       -8.0

In [53]:
change_rating(my_bonds_finam)

SECID  COUNT             SECNAME    LAST  LOTVALUE  YIELD  DURATION  \
0  RU000A10C8H9    1.0  Оил Ресурс 001P-02  100.08    1000.0  31.85       987   

  OFFERDATE     MATDATE  COUPONPERCENT  ACCRUEDINT  VOLTODAY RATING   VALUE  \
0      None  2030-07-06           28.0        5.37      1978      -  1000.8   

    PART  SELLCOUNT  
0  11.56        0.0

In [94]:
value_sum

np.float64(9615.599999999999)

In [95]:
my_bonds_finam['YIELD'].mean()

np.float64(28.08)

In [96]:
my_bonds_finam['COUNT'].sum()

np.float64(10.0)